Follow style guide https://www.python.org/dev/peps/pep-0008/

* Method Names and Instance Variables - Use the function naming rules: lowercase with words separated by underscores as necessary to improve readability.
* Use one leading underscore ONLY FOR NON PUBLIC methods and instance variables. e.g. _variable. note that private variables don't exist in Python. There are simply norms to be followed. 


* Constants are usually defined on a module level and written in all capital letters with underscores separating words. Examples include MAX_OVERFLOW and TOTAL


* Class names should normally use the CapWords convention.
* Always use self for the first argument to instance methods.
* Always use cls for the first argument to class methods.

Example https://github.com/home-assistant/home-assistant/blob/dev/homeassistant/components/sensor/wsdot.py

Understading @PROPERTY https://www.programiz.com/python-programming/property

In [1]:
## CONSTANTS
STANDARD_PRESSURE    = 760.0   
STANDARD_TEMPERATURE = 20.0    

In [2]:
class Chamber:
    '''Class for chambers'''
    def __init__(self, name, serial, cal_factor_dict):
        self._name = name
        self._serial = serial                   # could be a string or a number
        self._cal_factor = cal_factor_dict      # Nk, function of HVL but pass as dict
    
    def calibration(self, kV):                 # ignore fact that cal is in HVL for this exercise
        if kV not in self._cal_factor.keys():
            raise ValueError('No cal factor for that kV')
        return self._cal_factor[kV]

In [3]:
Farmer = Chamber('farmer', 3627, {80 : 4.143, 140 :4.143, 250 :4.079}) # init our farmer chamber

In [4]:
class RtMeasurement:
    '''Generic class of radiotherapy measurement. 
    Readings must be a list.
    Daily correction is the correction to apply to readings. E.g. if daily output is 2% low, this factor is 1.02
    Temperature in degrees Celcius, Pressure in mmHg.
    Field_type is rectangle, square or circle.
    Field x and y are in cm at the SSD.
    PDD is optionally entered and is on scale 0 - 100 %'''
    
    def __init__(self, chamber, readings, daily_correction, temperature, pressure, kV, MU, medium, depth, SSD, field_shape, field_x, field_y):
        self._chamber= chamber       # of class chamber
        self._readings = readings    # should be a list
        self._daily_correction = daily_correction
        self._temperature = temperature
        self._pressure = pressure
        self._kV = kV                 # should really be HVL
        self._MU = MU
        self._medium = medium         # i.e. air, phantom, water
        self._depth = depth           # depth in medium of measurement, e.g. 2 cm
        self._SSD = SSD               # source to surface distance, cm
        self._field_shape = field_shape # circle or rectangle
        self._field_x  = field_x
        self._field_y  = field_y      # field_x == field_y for a circle   
        self._ftp = ((self._temperature + (293.15 - STANDARD_TEMPERATURE))*STANDARD_PRESSURE)/(293.15*self._pressure)
        self._pdd = 1.0                 # Init default PDD
        self._scatter = 1.0             # Init default scatter factor, e.g. backscatter or Kch
        self._mass_energy = 1.0         # Init mass energy absorption coeff
    
    def add_reading(self, reading):
        self._readings.append(reading)
    
    def mean_reading(self):           # since new readings can be added, this must be calclated rather than an attribute
        return sum(self._readings)/len(self._readings)    # should be a list

    def corrected_mean_reading(self):
        return self.mean_reading()*self._daily_correction*self._ftp

    @property     # ftp is a property object which provides interface to the private variable _ftp. Use to access from outside class
    def ftp(self):    # getter
        return self._ftp
    
    @property     # ftp is a property object which provides interface to the private variable _ftp. Use to access from outside class
    def kV(self):    # getter
        return self._kV

    @property     # ftp is a property object which provides interface to the private variable _ftp. Use to access from outside class
    def MU(self):    # getter
        return self._MU

    @property
    def pdd(self):   # getter
        return self._pdd

    @pdd.setter      # Manually set, although should lookup
    def pdd(self, value):
        if value < 0.0 or value > 100.0:
            raise ValueError("Percent depth dose scale 0 - 100 ")
        print("Setting PDD")
        self._pdd = value

Would ideally create a calculations class

## Measurement 1: standard condition

Lets create measurement1

In [5]:
chamber = Farmer
readings =  [37.86, 37.88, 37.82] 
daily_correction = 1.0
temperature = 23.0
pressure = 760.8 
kV = 250 
MU = 200
medium = 'water'
depth =  2.0
SSD = 50
field_type = 'square'
field_x, field_y = 10.0, 10.0

kV250_meas_1 = RtMeasurement(chamber, readings, daily_correction, temperature, pressure, kV, MU, medium, depth, SSD, field_type, field_x, field_y)

print(kV250_meas_1.corrected_mean_reading())   # sanity check

38.200500755090275


Set the PDD for time being - should be calculated on init

In [6]:
kV250_meas_1.pdd = 88.0
print(kV250_meas_1.pdd)

Setting PDD
88.0


## Calculate daily output from standard conditions used in Measurement 1

In [7]:
def standard_output(reading, PDD, Nk, scatter_factor, mass_energy):
    return (100.0*reading/PDD)*Nk*scatter_factor*mass_energy

In [8]:
reading = kV250_meas_1.corrected_mean_reading()
PDD =  kV250_meas_1.pdd
Nk =  Farmer.calibration(kV250_meas_1.kV)     # Fetch cal factor

Kch =  1.019     # for this measurement, Kch                             
mass_energy = 1.090

kV250_meas_1_output = standard_output(reading, PDD, Nk, Kch, mass_energy)
print("kV250_meas_1_output is " + str(kV250_meas_1_output))

daily_output = kV250_meas_1_output/kV250_meas_1.MU
daily_output_correction = round((daily_output-1)*100.0, 2)

print("output per MU of " + str(daily_output)) 
print("Difference to standard output of " + str(daily_output_correction) + " %")

print("Daily output +/- 2% tolerance")

kV250_meas_1_output is 196.67120153641648
output per MU of 0.9833560076820824
Difference to standard output of -1.66 %
Daily output +/- 2% tolerance


## Measurement 2

As per measurement 1 but with a 15x15 applicator to calc the applicator factor as consistency check

In [9]:
readings =  [41.90, 41.80, 41.86]     # this is bad practice
field_x, field_y = 15.0, 15.0

kV250_meas_2 = RtMeasurement(chamber, readings, daily_correction, temperature, pressure, kV, MU, medium, depth, SSD, field_type, field_x, field_y)

In [10]:
PDD = 91.0
kV250_meas_2_output = standard_output(kV250_meas_2.corrected_mean_reading(), PDD, Nk, Kch, mass_energy)
print("Output 2 is " + str(kV250_meas_2_output))

Output 2 is 210.28484469770504


In [11]:
applicator_factor = kV250_meas_2_output/kV250_meas_1_output
print("applicator_factor is " + str(applicator_factor))

standard_applicator_factor = 1.08
diff_to_std_app = round(100.0*(applicator_factor-standard_applicator_factor)/standard_applicator_factor, 2)
print("Difference to standard applicator factor of " + str(diff_to_std_app) + " %")
print("Applicator factor check +/- 1% tolerance")

applicator_factor is 1.0692203182516673
Difference to standard applicator factor of -1.0 %
Applicator factor check +/- 1% tolerance


# Unused

In [12]:
class medium_kV_output(measurement):                 # subclass measurement and add backscatter
    '''Sub class measurement with kV, requires backscatter'''
    
    def __init__(self, chamber, readings, daily_correction, temperature, pressure, kV, medium, depth, backscatter_factor):
        super().__init__(chamber, readings, daily_correction, temperature, pressure, kV)   # init the superclass
        self.backscatter_factor = backscatter_factor
        
    def Get_Output(self):
        return self.Corrected_Mean_Reading()*self.backscatter_factor*self.Nk  # should add mass-energy              

NameError: name 'measurement' is not defined